In [115]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
# 항목을 고유 숫자로 라벨링함
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
from sklearn.neighbors import NearestNeighbors
from sklearn.neighbors import KNeighborsRegressor

In [89]:
area_a = pd.read_csv('../csv/capital/tn_visit_area_info_방문지정보_A.csv', low_memory=False)
area_b = pd.read_csv('../csv/east/tn_visit_area_info_방문지정보_B.csv', low_memory=False)
area_c = pd.read_csv('../csv/west/tn_visit_area_info_방문지정보_C.csv', low_memory=False)
area_d = pd.read_csv('../csv/jeju/tn_visit_area_info_방문지정보_D.csv', low_memory=False)

travel_a = pd.read_csv('../csv/capital/tn_travel_여행_A.csv', low_memory=False)
travel_b = pd.read_csv('../csv/east/tn_travel_여행_B.csv', low_memory=False)
travel_c = pd.read_csv('../csv/west/tn_travel_여행_C.csv', low_memory=False)
travel_d = pd.read_csv('../csv/jeju/tn_travel_여행_D.csv', low_memory=False)

traveller_a = pd.read_csv('../csv/capital/tn_traveller_master_여행객_Master_A.csv')
traveller_b = pd.read_csv('../csv/east/tn_traveller_master_여행객_Master_B.csv')
traveller_c = pd.read_csv('../csv/west/tn_traveller_master_여행객_Master_C.csv')
traveller_d = pd.read_csv('../csv/jeju/tn_traveller_master_여행객_Master_D.csv')

In [90]:
area_all = pd.concat([area_a, area_b, area_c, area_d], axis=0, ignore_index=True)
travel_all = pd.concat([travel_a, travel_b, travel_c, travel_d], axis=0, ignore_index=True)
traveller_all = pd.concat([traveller_a, traveller_b, traveller_c, traveller_d], axis=0, ignore_index=True)

In [91]:
area_all.to_csv('../csv/tn_visit_area_all.csv', index=False)
travel_all.to_csv('../csv/tn_visit_travel_all.csv', index=False)
traveller_all.to_csv('../csv/tn_visit_traveller_all.csv', index=False)

In [92]:
traveller_all.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12800 entries, 0 to 12799
Data columns (total 36 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   TRAVELER_ID                12800 non-null  object 
 1   RESIDENCE_SGG_CD           12800 non-null  int64  
 2   GENDER                     12800 non-null  object 
 3   AGE_GRP                    12800 non-null  int64  
 4   EDU_NM                     12800 non-null  int64  
 5   EDU_FNSH_SE                12799 non-null  float64
 6   MARR_STTS                  12798 non-null  float64
 7   FAMILY_MEMB                12800 non-null  int64  
 8   JOB_NM                     12227 non-null  float64
 9   JOB_ETC                    573 non-null    float64
 10  INCOME                     12800 non-null  int64  
 11  HOUSE_INCOME               9497 non-null   float64
 12  TRAVEL_TERM                12800 non-null  int64  
 13  TRAVEL_NUM                 12800 non-null  int

In [93]:
df = pd.merge(area_all, travel_all, how='left', on='TRAVEL_ID')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 164195 entries, 0 to 164194
Data columns (total 32 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   VISIT_AREA_ID         164195 non-null  int64  
 1   TRAVEL_ID             164195 non-null  object 
 2   VISIT_ORDER           164195 non-null  int64  
 3   VISIT_AREA_NM         164195 non-null  object 
 4   VISIT_START_YMD       164195 non-null  object 
 5   VISIT_END_YMD         164195 non-null  object 
 6   ROAD_NM_ADDR          117306 non-null  object 
 7   LOTNO_ADDR            129903 non-null  object 
 8   X_COORD               134598 non-null  object 
 9   Y_COORD               134598 non-null  object 
 10  ROAD_NM_CD            170 non-null     float64
 11  LOTNO_CD              10871 non-null   object 
 12  POI_ID                69397 non-null   object 
 13  POI_NM                69397 non-null   object 
 14  RESIDENCE_TIME_MIN    151341 non-null  float64
 15  

In [94]:
df = pd.merge(df, traveller_all, how='left', on='TRAVELER_ID')

In [95]:
df_filter = df[~df['TRAVEL_MISSION_CHECK'].isnull()].copy()  # 'TRAVEL_MISSION_CHECK' 컬럼에 결측치가 없는 행만을 복사

# 미션 = 여행목적으로 보여짐 한 컬럼에 최대 3개가 있지만 일단 맨 처음 1개만 사용
df_filter.loc[:, 'TRAVEL_MISSION_INT'] = df_filter['TRAVEL_MISSION_CHECK'].str.split(';').str[0].astype(int)

In [96]:
df_filter = df_filter[[
    'GENDER',  # 성별
    'AGE_GRP',  # 연령대
    # 여행스타일
    'TRAVEL_STYL_1', 'TRAVEL_STYL_2', 'TRAVEL_STYL_3', 'TRAVEL_STYL_4', 'TRAVEL_STYL_5', 'TRAVEL_STYL_6',
    'TRAVEL_STYL_7', 'TRAVEL_STYL_8',
    'TRAVEL_MOTIVE_1',  # 여행동기 (3까지 있음)
    'TRAVEL_COMPANIONS_NUM',  # 동반자 수
    'TRAVEL_MISSION_INT',  # 여행 목적 최우선순위
    'VISIT_AREA_NM',  # 방문지명
    'DGSTFN',  # 만족도
]]

df_filter = df_filter.dropna()

In [97]:
df_filter.info()

<class 'pandas.core.frame.DataFrame'>
Index: 86908 entries, 0 to 164194
Data columns (total 15 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   GENDER                 86908 non-null  object 
 1   AGE_GRP                86908 non-null  float64
 2   TRAVEL_STYL_1          86908 non-null  float64
 3   TRAVEL_STYL_2          86908 non-null  float64
 4   TRAVEL_STYL_3          86908 non-null  float64
 5   TRAVEL_STYL_4          86908 non-null  float64
 6   TRAVEL_STYL_5          86908 non-null  float64
 7   TRAVEL_STYL_6          86908 non-null  float64
 8   TRAVEL_STYL_7          86908 non-null  float64
 9   TRAVEL_STYL_8          86908 non-null  float64
 10  TRAVEL_MOTIVE_1        86908 non-null  float64
 11  TRAVEL_COMPANIONS_NUM  86908 non-null  float64
 12  TRAVEL_MISSION_INT     86908 non-null  int32  
 13  VISIT_AREA_NM          86908 non-null  object 
 14  DGSTFN                 86908 non-null  float64
dtypes: flo

In [98]:
df_filter

,GENDER,AGE_GRP,TRAVEL_STYL_1,TRAVEL_STYL_2,TRAVEL_STYL_3,TRAVEL_STYL_4,TRAVEL_STYL_5,TRAVEL_STYL_6,TRAVEL_STYL_7,TRAVEL_STYL_8,TRAVEL_MOTIVE_1,TRAVEL_COMPANIONS_NUM,TRAVEL_MISSION_INT,VISIT_AREA_NM,DGSTFN
0,남,30.0,4.0,4.0,2.0,4.0,6.0,5.0,6.0,6.0,1.0,0.0,5,프로방스마을,4.0
2,여,40.0,4.0,3.0,6.0,3.0,2.0,2.0,7.0,2.0,6.0,0.0,28,병점역 1호선,4.0
3,여,30.0,4.0,4.0,4.0,3.0,4.0,4.0,5.0,7.0,1.0,1.0,6,더현대서울,5.0
4,남,30.0,7.0,1.0,1.0,1.0,7.0,7.0,2.0,4.0,1.0,0.0,2,강릉중앙시장,4.0
5,남,20.0,6.0,1.0,1.0,3.0,4.0,6.0,3.0,2.0,3.0,1.0,11,청계천,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
164187,여,20.0,1.0,2.0,2.0,4.0,5.0,2.0,5.0,6.0,1.0,2.0,28,함덕촐래밥상,5.0
164189,남,30.0,3.0,3.0,3.0,4.0,3.0,3.0,4.0,6.0,2.0,1.0,26,문개항아리 조천본점,5.0
164192,남,40.0,2.0,2.0,5.0,1.0,1.0,3.0,2.0,3.0,1.0,1.0,2,메종글래드제주,3.0
164193,여,30.0,1.0,1.0,1.0,3.0,1.0,1.0,3.0,7.0,1.0,1.0,24,옹진농협하나로마트 덕적점,5.0


In [99]:
# 변환할 열 목록
columns_to_convert = ['AGE_GRP', 'TRAVEL_STYL_1', 'TRAVEL_STYL_2', 'TRAVEL_STYL_3',
                      'TRAVEL_STYL_4', 'TRAVEL_STYL_5', 'TRAVEL_STYL_6', 'TRAVEL_STYL_7',
                      'TRAVEL_STYL_8', 'TRAVEL_MOTIVE_1', 'TRAVEL_COMPANIONS_NUM', 'DGSTFN']

# 성별도 int형으로 변환
df_filter['GENDER'] = df_filter['GENDER'].replace({'남': 1, '여': 0}).astype('int32')

# integer로 변환 (int32 사용으로 메모리 절약)
df_filter[columns_to_convert] = df_filter[columns_to_convert].astype('int32')

df_filter.dtypes


C:\Users\user\AppData\Local\Temp\ipykernel_21388\439395396.py:7: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_filter['GENDER'] = df_filter['GENDER'].replace({'남': 1, '여': 0}).astype('int32')


GENDER                    int32
AGE_GRP                   int32
TRAVEL_STYL_1             int32
TRAVEL_STYL_2             int32
TRAVEL_STYL_3             int32
TRAVEL_STYL_4             int32
TRAVEL_STYL_5             int32
TRAVEL_STYL_6             int32
TRAVEL_STYL_7             int32
TRAVEL_STYL_8             int32
TRAVEL_MOTIVE_1           int32
TRAVEL_COMPANIONS_NUM     int32
TRAVEL_MISSION_INT        int32
VISIT_AREA_NM            object
DGSTFN                    int32
dtype: object

In [111]:
le = LabelEncoder()

# 숫자로 방문지명 라벨링
encoded_area = le.fit_transform(df_filter['VISIT_AREA_NM'])

In [101]:
X = df_filter.drop(columns=['DGSTFN'])
X['VISIT_AREA_NM'] = encoded_area

Y = df_filter['DGSTFN']
visit_areas = df_filter['VISIT_AREA_NM']

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=42)

In [102]:
rd = RandomForestClassifier()
y_pred = rd.fit(X_train, y_train)

In [103]:
rd_score = rd.score(X_test, y_test)
rd_score

0.5366087523491735

In [104]:
y_pred = rd.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
mse

1.0461780385839758

In [112]:
knn = KNeighborsClassifier(n_neighbors=100)
knn.fit(X_train, y_train)

KNeighborsClassifier(n_neighbors=100)

In [113]:
knn_score = knn.score(X_test, y_test)
knn_score

0.4553752924481264

In [107]:
nn = NearestNeighbors(n_neighbors=20, metric='euclidean')
nn.fit(X_train, y_train)

NearestNeighbors(metric='euclidean', n_neighbors=20)

-0.022408679934187603